# Supervised Machine Learning: Water Treatment Plant 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Data Collection

In [2]:
url=f"https://archive.ics.uci.edu/ml/machine-learning-databases/water-treatment/water-treatment.data"

In [5]:
df=pd.read_csv(url, header=None)

In [35]:
names=f"""Date   (Date D/M/YY)
Q-E        (input flow to plant)  
ZN-E       (input Zinc to plant)
PH-E       (input pH to plant) 
DBO-E      (input Biological demand of oxygen to plant) 
DQO-E      (input chemical demand of oxygen to plant)
SS-E       (input suspended solids to plant)  
SSV-E      (input volatile supended solids to plant)
SED-E      (input sediments to plant) 
COND-E     (input conductivity to plant) 
PH-P       (input pH to primary settler)
DBO-P      (input Biological demand of oxygen to primary settler)
SS-P       (input suspended solids to primary settler)
SSV-P      (input volatile supended solids to primary settler)
SED-P      (input sediments to primary settler) 
COND-P     (input conductivity to primary settler)
PH-D       (input pH to secondary settler) 
DBO-D      (input Biological demand of oxygen to secondary settler)
DQO-D      (input chemical demand of oxygen to secondary settler)
SS-D       (input suspended solids to secondary settler)
SSV-D      (input volatile supended solids to secondary settler)
SED-D      (input sediments to secondary settler)  
COND-D     (input conductivity to secondary settler) 
PH-S       (output pH)   
DBO-S      (output Biological demand of oxygen)
DQO-S      (output chemical demand of oxygen)
SS-S       (output suspended solids)
SSV-S      (output volatile supended solids) 
SED-S      (output sediments) 
COND-S     (output conductivity)
RD-DBO-P   (performance input Biological demand of oxygen in primary settler)
RD-SS-P    (performance input suspended solids to primary settler)
RD-SED-P   (performance input sediments to primary settler)
RD-DBO-S   (performance input Biological demand of oxygen to secondary settler)
RD-DQO-S   (performance input chemical demand of oxygen to secondary settler)
RD-DBO-G   (global performance input Biological demand of oxygen)
RD-DQO-G   (global performance input chemical demand of oxygen)
RD-SS-G    (global performance input suspended solids) 
RD-SED-G   (global performance input sediments)"""

In [36]:
names = [[i.strip(" ") for i in row.strip(r" )").split(r"  (")] for row in names.split(f"\n")]

In [38]:
df.columns=pd.DataFrame(names)[0]

In [39]:
df.head()

,Date,Q-E,ZN-E,PH-E,DBO-E,DQO-E,SS-E,SSV-E,SED-E,COND-E,...,COND-S,RD-DBO-P,RD-SS-P,RD-SED-P,RD-DBO-S,RD-DQO-S,RD-DBO-G,RD-DQO-G,RD-SS-G,RD-SED-G
0,D-1/3/90,44101,1.50,7.8,?,407,166,66.3,4.5,2110,...,2000,?,58.8,95.5,?,70.0,?,79.4,87.3,99.6
1,D-2/3/90,39024,3.00,7.7,?,443,214,69.2,6.5,2660,...,2590,?,60.7,94.8,?,80.8,?,79.5,92.1,100
2,D-4/3/90,32229,5.00,7.6,?,528,186,69.9,3.4,1666,...,1888,?,58.2,95.6,?,52.9,?,75.8,88.7,98.5
3,D-5/3/90,35023,3.50,7.9,205,588,192,65.6,4.5,2430,...,1840,33.1,64.2,95.3,87.3,72.3,90.2,82.3,89.6,100
4,D-6/3/90,36924,1.50,8.0,242,496,176,64.8,4.0,2110,...,2120,?,62.7,95.6,?,71.0,92.1,78.2,87.5,99.5
